# sllm 모델 실험
- 1. trillion 7B
- 2. Linkbricks Korean model
- 3. Blossom 5B

In [1]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.6/486.6 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 5.1.0
    Uninstalling sentence-transformers-5.1.0:
      Successfully uninstalled sentence-transformers-5.1.0


In [2]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 73.9 MB/s eta 0:00:00


In [3]:
!pip install einops

In [4]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


### 1. 임베딩 모델 선정

In [5]:
from sentence_transformers import SentenceTransformer
from langchain_community.embeddings import HuggingFaceEmbeddings

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings_model = HuggingFaceEmbeddings(
    model_name = 'all-MiniLM-L6-v2',
    model_kwargs = {'device':'cpu'},
    encode_kwargs = {'normalize_embeddings':True}
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipython-input-1066112194.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(


### 2. vector store 설정


In [6]:
from langchain.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
import faiss
from langchain_community.document_loaders import CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

loader = CSVLoader(file_path="전국 데이터.csv", encoding="utf-8")
pages = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

vectorstore = FAISS.from_documents(docs, embeddings_model)

### 3. 검색해보자(예시)

In [ ]:
import pandas as pd

df = pd.read_csv('전국 데이터.csv')

In [ ]:
# 검색 쿼리
query = '가족들과 함께 가기 좋은 계곡'

# 가장 유사도가 높은 문장을 하나만 추출
retriever = vectorstore.as_retriever(search_kwargs={'k': 5},search_type = 'similarity')

docs = retriever.get_relevant_documents(query)

print(len(docs))
for i in range(len(docs)):
  print(f'{i+1}번째 : .{df.iloc[docs[i].metadata['row']]['place']}')

5
1번째 : .팔복예술공장
2번째 : .소노캄 제주
3번째 : .망원한강공원
4번째 : .섭지코지
5번째 : .완산벙커 더 스페이스


## 벡터db 저장


In [8]:
from google.colab import files

vectorstore.save_local('FAISS(500,0,L6-v2)')
files.download('FAISS(500,0,L6-v2)')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
from google.colab import drive
drive.mount('/content/drive')
# Copy your FAISS index file from Google Drive to the Colab environment
# !cp "/content/drive/My Drive/your_faiss_index.faiss" "/content/your_faiss_index.faiss"

Mounted at /content/drive


---
# 본격적인 플로우 생성

In [9]:
# 프롬프트 설정
from langchain_core.prompts import ChatPromptTemplate

messages = [('system', '''당신은 대한민국 관광지에 대해 잘 알려주는 친절한 챗봇입니다. 사용자가 원하는 답변을 5문장 이내로 해줍시다.'''),
    ('user','''context만을 이용해 답변하세요. context에서 확인할 수 없는 질문이라면 모른다고 답변해야합니다.

    질문 : {query}
    contenxt = {context}

    <<최종 응답 형식>>
    답변:
    참조문서 :
    <<source>>  (page : <<page>>): <page_context>'''

    )]
# 대화 내용 기억하도록 messages를 업데이트 하는 함수
def update_history(messages,response):
  if len(messages) %2 ==0:
    messages.append(('system',response))
  else:
    messages.append(('user',response))
  return messages

prompt_template = ChatPromptTemplate(messages)

In [10]:
# sllm 모델 생성
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "trillionlabs/Trillion-7B-preview"

sllm = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--trillionlabs--Trillion-7B-preview/snapshots/bc1650c5a954c7de0509d28c8dbc5bea560856e1/config.json
`torch_dtype` is deprecated! Use `dtype` instead!
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "dtype": "bfloat16",
  "eos_token_id": 128001,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 100000.0,
  "tie_word_embeddings": false,
  "transformers_version": "4.56.1",
  "use_cache": true,
  "vocab_size": 128128
}



model.safetensors.index.json: 0.00B [00:00, ?B/s]

loading weights file model.safetensors from cache at /root/.cache/huggingface/hub/models--trillionlabs--Trillion-7B-preview/snapshots/bc1650c5a954c7de0509d28c8dbc5bea560856e1/model.safetensors.index.json


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

...

loading configuration file generation_config.json from cache at /root/.cache/huggingface/hub/models--trillionlabs--Trillion-7B-preview/snapshots/bc1650c5a954c7de0509d28c8dbc5bea560856e1/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 0,
  "do_sample": true,
  "eos_token_id": 128001,
  "pad_token_id": 128004,
  "temperature": 0.6,
  "top_p": 0.9
}



tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--trillionlabs--Trillion-7B-preview/snapshots/bc1650c5a954c7de0509d28c8dbc5bea560856e1/tokenizer.json
loading file tokenizer.model from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--trillionlabs--Trillion-7B-preview/snapshots/bc1650c5a954c7de0509d28c8dbc5bea560856e1/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--trillionlabs--Trillion-7B-preview/snapshots/bc1650c5a954c7de0509d28c8dbc5bea560856e1/tokenizer_config.json
loading file chat_template.jinja from cache at None


In [14]:
# chain 생성
from langchain_core.output_parsers import StrOutputParser

chain = prompt_template | sllm | StrOutputParser

In [21]:
# 검색 쿼리
query = "가족들과 가기 좋은 자연 속 관광지 추천해줘."

# Retriever 생성
retriever = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={'k': 10, 'lambda_mult': 0.15}   # 상위 10개 뽑아주기
)

retriever_result = retriever.batch([query])
# retrieval_result = retriever.get_relevant_documents(query)
# response = chain.invoke({'query':query,'context':retriever_result}) # Remove this line

print(retriever_result)

prompt = prompt_template.invoke({'query':query,'context':retriever_result})
model_inputs = tokenizer([prompt.to_string()], return_tensors="pt").to(sllm.device) # Use sllm.device

generated_ids = sllm.generate( # Use sllm.generate
    model_inputs["input_ids"],
    attention_mask=model_inputs["attention_mask"],
    max_new_tokens=1024
)

[[Document(id='789bf7e0-ff95-4c4f-a78d-d1860a085f40', metadata={'source': '전국 데이터.csv', 'row': 5517}, page_content='구간에 미소공원과 갯바람공원, 일몰공원이 속해 있다.'), Document(id='6175e943-6615-40e2-88a4-ebbf16081029', metadata={'source': '전국 데이터.csv', 'row': 9672}, page_content='실권과 국권의 회복문제를 국제여론에 호소하려다 실패한 후 이준은 현지에서 순사 하였다.  (출처: 진천군 문화관광 홈페이지)'), Document(id='4913753a-194d-4d20-9bb4-855a3038ef96', metadata={'source': '전국 데이터.csv', 'row': 8852}, page_content='주변의 여행지와 함께 묶어서 방문하는 것을 권장한다.'), Document(id='35e3809d-5643-48dc-b10d-5a9e4594d925', metadata={'source': '전국 데이터.csv', 'row': 18550}, page_content='진정작용, 신진대사, 노화 방지에도 효과가 있는 것으로 알려져 찾는 이들이 많다.'), Document(id='a7f1df8a-4f6e-448a-87f3-c3829026d0a8', metadata={'source': '전국 데이터.csv', 'row': 1295}, page_content='부산포개항문화관, 동구도서관, 자성대 공원, 만리산 체육공원, 전포 돌산 공원, 부산 중앙 부두 등이 있다.'), Document(id='955d43b7-be28-47be-ab6a-507a0bcdc310', metadata={'source': '전국 데이터.csv', 'row': 8074}, page_content='홍주성역사공원, 홍주성역사관이 있어 연계해서 둘러보는 것도 좋다.'), Document(id='76baee94

In [22]:
print(retriever_result)

[[Document(id='789bf7e0-ff95-4c4f-a78d-d1860a085f40', metadata={'source': '전국 데이터.csv', 'row': 5517}, page_content='구간에 미소공원과 갯바람공원, 일몰공원이 속해 있다.'), Document(id='6175e943-6615-40e2-88a4-ebbf16081029', metadata={'source': '전국 데이터.csv', 'row': 9672}, page_content='실권과 국권의 회복문제를 국제여론에 호소하려다 실패한 후 이준은 현지에서 순사 하였다.  (출처: 진천군 문화관광 홈페이지)'), Document(id='4913753a-194d-4d20-9bb4-855a3038ef96', metadata={'source': '전국 데이터.csv', 'row': 8852}, page_content='주변의 여행지와 함께 묶어서 방문하는 것을 권장한다.'), Document(id='35e3809d-5643-48dc-b10d-5a9e4594d925', metadata={'source': '전국 데이터.csv', 'row': 18550}, page_content='진정작용, 신진대사, 노화 방지에도 효과가 있는 것으로 알려져 찾는 이들이 많다.'), Document(id='a7f1df8a-4f6e-448a-87f3-c3829026d0a8', metadata={'source': '전국 데이터.csv', 'row': 1295}, page_content='부산포개항문화관, 동구도서관, 자성대 공원, 만리산 체육공원, 전포 돌산 공원, 부산 중앙 부두 등이 있다.'), Document(id='955d43b7-be28-47be-ab6a-507a0bcdc310', metadata={'source': '전국 데이터.csv', 'row': 8074}, page_content='홍주성역사공원, 홍주성역사관이 있어 연계해서 둘러보는 것도 좋다.'), Document(id='76baee94

In [23]:
generated_ids = [
    output_ids[len(input_inputs):] for input_inputs, output_ids in zip(model_inputs.input_ids, generated_ids) # Correct variable name
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)


    <<source>>  (page : <<page>>): <page_context>
    <<source>>  (page : <<page>>): <page_context>
    <<source>>  (page : <<page>>): <page_context>
    <<source>>  (page : <<page>>): <page_context>

    <<최종 답변>>
    답변: 자연 속 관광지를 찾고자 한다면 진천의 농다리 공원을 추천합니다. 근처에 초평저수지와 초평 붕어마을 등 다양한 자연 관광지가 있으며, 농다리 축제 등 다양한 행사도 진행되고 있습니다. 농다리는 고려시대 때 축조된 것으로 알려져 있으며, 교각과 교각 사이의 지점에 구멍이 있어 그 구멍으로 물이 흘러가는 독특한 다리입니다. 진천 농교라는 명칭으로 충청북도의 유형문화재로 지정되었습니다. (출처: 진천군 문화관광 홈페이지)

    <<최종 답변>>
    답변: 자연 속 관광지를 찾고자 한다면 진천의 농다리 공원을 추천합니다. 근처에 초평저수지와 초평 붕어마을 등 다양한 자연 관광지가 있으며, 농다리 축제 등 다양한 행사도 진행되고 있습니다. 농다리는 고려시대 때 축조된 것으로 알려져 있으며, 교각과 교각 사이의 지점에 구멍이 있어 그 구멍으로 물이 흘러가는 독특한 다리입니다. 진천 농교라는 명칭으로 충청북도의 유형문화재로 지정되었습니다. (출처: 진천군 문화관광 홈페이지)

    <<최종 답변>>
    답변: 자연 속 관광지를 찾고자 한다면 진천의 농다리 공원을 추천합니다. 근처에 초평저수지와 초평 붕어마을 등 다양한 자연 관광지가 있으며, 농다리 축제 등 다양한 행사도 진행되고 있습니다. 농다리는 고려시대 때 축조된 것으로 알려져 있으며, 교각과 교각 사이의 지점에 구멍이 있어 그 구멍으로 물이 흘러가는 독특한 다리입니다. 진천 농교라는 명칭으로 충청북도의 유형문화재로 지정되었습니다. (출처: 진천군 문화관광 홈페이지)

```

```python
# 필요한 라이브